In [11]:
#!pip install nltk
#!pip install gensim

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import gensim
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
import gensim.downloader as api
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [6]:
df_IoT=pd.read_pickle("./data/IoT_data/IotData.pkl")
df_kdd=pd.read_pickle("./data/IoT_data/kddData.pkl")

In [7]:
df_IoT.head()

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc,type
208516,180.436572,247.237321,5.742339e+04,297.004194,273.323803,5.985489e+04,718.666602,312.743438,60956.478168,5094.993410,...,0.000000e+00,0.0,1.000000,60.0,0.0,60.000000,0.0,0.000000e+00,0.0,attack
39451,2.988515,102.000000,3.640000e-12,2.993097,102.000000,3.180000e-10,2.997779,101.998857,0.047875,4.215173,...,3.920000e-35,0.0,4.219798,102.0,0.0,144.249783,0.0,1.130000e-30,0.0,benign
146186,94.513273,527.164570,1.814254e+04,173.339387,485.307159,3.431924e+04,594.124501,425.632650,51322.193965,6286.184621,...,0.000000e+00,0.0,1.000000,566.0,0.0,566.000000,0.0,0.000000e+00,0.0,attack
194097,111.238814,71.594983,2.788613e+01,199.017191,70.530434,3.653603e+01,642.620016,69.624159,42.113864,5733.591525,...,0.000000e+00,0.0,1.000000,74.0,0.0,74.000000,0.0,0.000000e+00,0.0,attack
140015,109.977805,459.069582,4.267268e+04,197.536848,424.200461,5.164345e+04,626.827988,397.930813,56794.199568,6345.800304,...,0.000000e+00,0.0,1.000000,566.0,0.0,566.000000,0.0,0.000000e+00,0.0,attack


In [8]:
df_kdd.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,target,Attack Type
395597,0,tcp,kshell,S0,0,0,0,0,0,0,...,0.04,0.05,0.0,0.0,1.0,1.0,0.0,0.0,neptune.,dos
117469,0,tcp,private,S0,0,0,0,0,0,0,...,0.03,0.07,0.0,0.0,1.0,1.0,0.0,0.0,neptune.,dos
179797,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,1.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,smurf.,dos
421060,0,icmp,ecr_i,SF,520,0,0,0,0,0,...,1.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,smurf.,dos
167145,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,1.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,smurf.,dos


In [29]:
model = api.load('glove-twitter-25')

def get_vector(word):
    try:
        return model[word]
    except KeyError:
        # Return a zero vector if the word is not in the vocabulary
        return np.zeros(model.vector_size)

In [30]:
df_kdd['pt_vec'] = df_kdd['protocol_type'].apply(get_vector)
df_kdd['pt_svc'] = df_kdd['service'].apply(get_vector)
df_kdd['pt_flg'] = df_kdd['flag'].apply(get_vector)
df_kdd['pt_tgt'] = df_kdd['target'].apply(get_vector)

In [31]:
cols = df_kdd.columns.tolist()  # Get a list of all column names
cols.remove('Attack Type')            # Remove 'B' from the list
cols.append('Attack Type')            # Add 'B' to the end of the list
df_kdd = df_kdd[cols]
df_kdd.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,target,pt_vec,pt_svc,pt_flg,pt_tgt,Attack Type
395597,0,tcp,kshell,S0,0,0,0,0,0,0,...,1.0,1.0,0.0,0.0,neptune.,"[0.43085, 0.76283, -0.31915, -0.13596, 0.04237...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",dos
117469,0,tcp,private,S0,0,0,0,0,0,0,...,1.0,1.0,0.0,0.0,neptune.,"[0.43085, 0.76283, -0.31915, -0.13596, 0.04237...","[-0.054061, 0.80721, -0.20382, -0.39667, 0.982...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",dos
179797,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,0.0,0.0,0.0,0.0,smurf.,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",dos
421060,0,icmp,ecr_i,SF,520,0,0,0,0,0,...,0.0,0.0,0.0,0.0,smurf.,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",dos
167145,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,0.0,0.0,0.0,0.0,smurf.,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",dos


In [34]:
df_kdd = df_kdd.drop('protocol_type', axis=1)
df_kdd = df_kdd.drop('service', axis=1)
df_kdd = df_kdd.drop('flag', axis=1)
df_kdd = df_kdd.drop('target', axis=1)
df_kdd.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,pt_vec,pt_svc,pt_flg,pt_tgt,Attack Type
395597,0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,"[0.43085, 0.76283, -0.31915, -0.13596, 0.04237...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",dos
117469,0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,"[0.43085, 0.76283, -0.31915, -0.13596, 0.04237...","[-0.054061, 0.80721, -0.20382, -0.39667, 0.982...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",dos
179797,0,1032,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",dos
421060,0,520,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",dos
167145,0,1032,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",dos


In [46]:
def calculate_mean(vector):
    return np.mean(vector)

# Apply the function to the 'vectors' column
df_kdd['pt_vec'] = df_kdd['pt_vec'].apply(calculate_mean)
df_kdd['pt_svc'] = df_kdd['pt_svc'].apply(calculate_mean)
df_kdd['pt_flg'] = df_kdd['pt_flg'].apply(calculate_mean)
df_kdd['pt_tgt'] = df_kdd['pt_tgt'].apply(calculate_mean)

In [47]:
df_kdd.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,pt_vec,pt_svc,pt_flg,pt_tgt,Attack Type
395597,0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,0.104219,0.000000,0.0,0.0,dos
117469,0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,0.104219,-0.205553,0.0,0.0,dos
179797,0,1032,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,dos
421060,0,520,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,dos
167145,0,1032,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,dos


In [52]:
df_kdd['Attack Type'] = (df_kdd['Attack Type'] != 'normal').astype(int)
df_kdd.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,pt_vec,pt_svc,pt_flg,pt_tgt,Attack Type
395597,0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,0.104219,0.000000,0.0,0.0,1
117469,0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,0.104219,-0.205553,0.0,0.0,1
179797,0,1032,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,1
421060,0,520,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,1
167145,0,1032,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,1


In [59]:
# Standard Scaler
scaler_standard = StandardScaler()
df_kdd_dropped_AT = df_kdd.drop('Attack Type', axis=1)
df_kdd_standard_scaled = pd.DataFrame(scaler_standard.fit_transform(df_kdd_dropped_AT), columns=df_kdd_dropped_AT.columns)
df_kdd_standard_scaled['Attack Type'] = df_kdd['Attack Type']

sampler=np.random.permutation(len(df_kdd_standard_scaled))
df_kdd_standard_scaled=df_kdd_standard_scaled.take(sampler)

df_kdd_standard_scaled.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,pt_vec,pt_svc,pt_flg,pt_tgt,Attack Type
454816,-0.067792,-0.002017,-0.026287,-0.006673,-0.04772,-0.002571,-0.044136,-0.009782,-0.417192,-0.005679,...,-0.158629,-0.464418,-0.463202,-0.25204,-0.249464,-0.821297,0.742168,0.0,0.066928,1
96364,-0.067792,-0.002559,-0.017843,-0.006673,-0.04772,-0.002571,-0.044136,-0.009782,2.396980,-0.005679,...,-0.158629,-0.464418,-0.463202,-0.25204,-0.249464,0.955787,1.435111,0.0,0.066928,1
454549,-0.067792,-0.002017,-0.026287,-0.006673,-0.04772,-0.002571,-0.044136,-0.009782,-0.417192,-0.005679,...,-0.158629,-0.464418,-0.463202,-0.25204,-0.249464,-0.821297,0.742168,0.0,0.066928,1
195889,-0.067792,-0.002017,-0.026287,-0.006673,-0.04772,-0.002571,-0.044136,-0.009782,-0.417192,-0.005679,...,-0.158629,-0.464418,-0.463202,-0.25204,-0.249464,-0.821297,0.742168,0.0,0.066928,1
437071,-0.067792,-0.002777,0.031310,-0.006673,-0.04772,-0.002571,-0.044136,-0.009782,2.396980,-0.005679,...,-0.158629,-0.464418,-0.463202,-0.25204,-0.249464,0.955787,-1.086324,0.0,0.066928,1


In [56]:
df_IoT['type'] = (df_IoT['type'] != 'benign').astype(int)
df_IoT.head()

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc,type
208516,180.436572,247.237321,5.742339e+04,297.004194,273.323803,5.985489e+04,718.666602,312.743438,60956.478168,5094.993410,...,0.000000e+00,0.0,1.000000,60.0,0.0,60.000000,0.0,0.000000e+00,0.0,1
39451,2.988515,102.000000,3.640000e-12,2.993097,102.000000,3.180000e-10,2.997779,101.998857,0.047875,4.215173,...,3.920000e-35,0.0,4.219798,102.0,0.0,144.249783,0.0,1.130000e-30,0.0,0
146186,94.513273,527.164570,1.814254e+04,173.339387,485.307159,3.431924e+04,594.124501,425.632650,51322.193965,6286.184621,...,0.000000e+00,0.0,1.000000,566.0,0.0,566.000000,0.0,0.000000e+00,0.0,1
194097,111.238814,71.594983,2.788613e+01,199.017191,70.530434,3.653603e+01,642.620016,69.624159,42.113864,5733.591525,...,0.000000e+00,0.0,1.000000,74.0,0.0,74.000000,0.0,0.000000e+00,0.0,1
140015,109.977805,459.069582,4.267268e+04,197.536848,424.200461,5.164345e+04,626.827988,397.930813,56794.199568,6345.800304,...,0.000000e+00,0.0,1.000000,566.0,0.0,566.000000,0.0,0.000000e+00,0.0,1


In [60]:
# Standard Scaler
scaler_standard = StandardScaler()
df_IoT_dropped_AT = df_IoT.drop('type', axis=1)
df_IoT_standard_scaled = pd.DataFrame(scaler_standard.fit_transform(df_IoT_dropped_AT), columns=df_IoT_dropped_AT.columns)
df_IoT_standard_scaled['Attack Type'] = df_IoT['type']

sampler=np.random.permutation(len(df_IoT_standard_scaled))
df_IoT_standard_scaled=df_IoT_standard_scaled.take(sampler)

df_IoT_standard_scaled.head()

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc,Attack Type
49533,0.343251,2.437210,0.063571,0.507774,2.235071,0.620085,0.756944,1.827138,1.220649,0.996901,...,-0.011188,-0.016171,-0.184341,2.036068,-0.025337,2.011124,-0.012603,-0.01115,-0.032123,0
93899,-1.148067,-0.439153,-0.613534,-1.149546,-0.459583,-0.639390,-1.142232,-0.477132,-0.653180,-1.135319,...,-0.011188,-0.016171,-0.183005,-0.350359,-0.025336,-0.189836,-0.012603,-0.01115,-0.032123,1
56100,-1.119081,-0.411869,-0.613534,-1.131234,-0.431286,-0.639390,-1.135898,-0.448229,-0.653180,-1.134690,...,-0.011188,-0.016171,-0.183341,-0.329426,-0.025337,-0.159864,-0.012603,-0.01115,-0.032123,1
188917,-1.148067,-0.698352,-0.613534,-1.149546,-0.728404,-0.639390,-1.142232,-0.751711,-0.653180,-1.135659,...,-0.011188,-0.016171,-0.184341,-0.549228,-0.025337,-0.606246,-0.012603,-0.01115,-0.032123,1
60741,1.264267,0.539349,1.821256,1.188261,0.916127,1.794928,1.026018,1.364279,1.602694,1.008771,...,-0.011188,-0.016171,-0.184341,-0.549228,-0.025337,-0.606246,-0.012603,-0.01115,-0.032123,1


In [61]:
df_kdd_standard_scaled.to_pickle("./data/IoT_data/kddDataClean.pkl")
df_IoT_standard_scaled.to_pickle("./data/IoT_data/IotDataClean.pkl")